In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
import matplotlib.pyplot as plt

In [9]:
class Portfolio:
    ## This class is an additional class to support the portfolio trading. Within this class, it would be possible to add portfolio Monitoring
    ## functionalities. Please Note that this class is only for the support of all Trading classes.
    def __init__(self):

        ## get the stock names
        self.s1_name = ''
        self.s2_name = ''

        self.s1_amount = 0
        self.s2_amount = 0
        self.money = 0
        self.pf_value =  ''
        self.sharpe = ''
        self.open_days = []
        self.avg_open_days = ''

    def sharpe_ratio(self):
        returns = self.pf_value.pct_change()
        returns = returns[1:]
        self.sharpe = returns.mean()/returns.std()

    def average_open_days(self):
        if len(self.open_days) == 0:
            self.avg_open_days = 0
        else:
            self.avg_open_days = sum(self.open_days)/len(self.open_days)


In [ ]:
class RegularTrading:
    ## This is the first trading class for the thesis. It only supports Regular trading without the addition of a Stoploss or potential last day trading.
    ## Usecase of this class is the initial Start of the analysis for the individual timeframes. The functions within the class
    ## are explained where necessary.

    def __init__(self, start_date, end_date):
        ## setting start and end date of the crisis
        self.start_date = start_date
        self.end_date = end_date

        ## setting datapoints of the trading
        self.tickers = ''
        self.data = ''
        self.normed_data = ''

        self.trading_data = {}

        self.distance_data = ''
        self.run()


    def run(self):
        ## This method is used to auto-initialize the RegularTrading Class. With its help, we can focus on the important stuff - the analysis.

        self.tickers = self.get_ticker()
        self.data = self.get_ticker_data()
        self.normed_data = self.normer(self.data)
        self.distance_data = self.calc_distance(self.normed_data)
        self.perform_distance_trading()

    def get_ticker(self):

        ##This method is a function to return all stock picks. With its return of the Stock picks, we can easliy track
        ## which tickers have been traded.

        companies = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        tickers = companies[0].Symbol.to_list()
        random.seed(89146)
        stock_picks = random.sample(tickers, k = 25)
        return stock_picks

    def get_ticker_data(self):

        ## This is the download function to get all the ticker data from yfinance. For this usage, only the
        ## Adjusted Close will be downloaded.

        df = yf.download(self.tickers, self.start_date, self.end_date)['Adj Close']
        return df


    def train_split(self, data, length):

        ## This method is seperating the dataframe into the train period.

        train_period = pd.Series(dtype= 'float64')
        train_period = data.iloc[:int(len(data)*length)]
        return train_period

    def test_split(self, data, length):

        ## This method is seperating the dataframe into the test period.

        test_period = pd.Series(dtype= 'float64')
        test_period = data.iloc[int(len(data)*length):]
        return test_period

    def data_split(self, data, thres):
        ## This method takes the input data and splits it to two dataframes: train and test.
        train = self.train_split(data, thres)
        test = self.test_split(data, thres)
        return train, test

    def normer(self, data):

        ##The normer method (as the name suggests) divides the input data with the first price from the input timeseries and returns the normalized data"""
        normed_data = data/data.iloc[0]
        return normed_data

    def calc_distance(self, data):

        ## The calc_distance method calculates the euclidean distance from all timeseries of the normed data. It lists them in an ascending order
        ## in the dataframe - this is a crucial step in the processing of the output."""
        df = pd.DataFrame()
        for a1 in data.columns:
            for a2 in data.columns:

                if a1 != a2:

                    ## using the euclidean distance
                    diff = sum((data[a1] - data[a2])**2)

                    ## the calculated data will be appended to the dataframe with the series s below:

                    s = pd.Series({
                        'stock1': a1,
                        'stock2': a2,
                        'diff': diff
                    })

                    df = pd.concat([df, s.to_frame().T], ignore_index = True)

        df = df.drop_duplicates(subset = 'diff')
        df = df.sort_values(by = 'diff', ascending = True)
        df = df.reset_index()
        df.drop('index', axis = 1, inplace = True)

        return df

    def distance_trade_prep(self, stock1, stock2):

        ## This method prepares the input of the stocks (stock1 and stock2) for the further trading in the process. It will be used for the formation period
        ## of the trading. Therefore, the return of the method are the normed train data as well as the mean and the standard deviation.

        s = pd.Series(dtype= 'float64')


        train_s1, _ = self.data_split(stock1, 0.75)
        train_s2, _ = self.data_split(stock2, 0.75)

        normed_s1 = self.normer(train_s1)
        normed_s2 = self.normer(train_s2)

        s = normed_s1 - normed_s2

        mean = s.mean()
        std = s.std()

        return mean, std

    def pre_trade(self, stock1, stock2, mean, std, p: Portfolio()):

        ## This method will prepare everything for the trading itself. The returns are the difference of the trading data, the price
        ## data for the two stocks, the mean, std-deviation and the start portfolio which contains an
        ## stock1 amount of 0, stock2 amount of 0 and 0 money. This all is achived by combining the previous methods into the function.

        normed_s1 = self.normer(stock1)
        normed_s2 = self.normer(stock2)

        _, trade_normed1 = self.data_split(data = normed_s1,thres= 0.75)
        _, trade_normed2 = self.data_split(data = normed_s2, thres= 0.75)

        _, trade1 = self.data_split(data = stock1, thres = 0.75)
        _, trade2 = self.data_split(data = stock2, thres = 0.75)

        diff_trade = trade_normed1 - trade_normed2

        p.s1_name = stock1.name
        p.s2_name = stock2.name

        p.s1_amount = 0
        p.s2_amount = 0
        p.money = 0.01

        mean = mean
        std = std

        return diff_trade, trade1, trade2, mean, std, p



    def trading(self, diff_trade, trade1, trade2, mean, std, thres, p: Portfolio()):

        ## The Trading Function is spectacular unspectacular - it uses the provided inputs (stock1, stock2, pre-calculated mean, pre-calculated std,
        ## the threshold for the trading signals and the portfolio itself. Within the RegularTrading Class,
        ## the inclusion of the portfolio doesn't have o much impact, but in the later stages (and Classes) it will show its benefits.

        df = pd.DataFrame()

        thres = thres


        diff_trade = diff_trade

        mean = mean
        std = std
        thres = thres

        y = 0

        for i in range(len(diff_trade)):

            portfolio_val = -(p.s1_amount * trade1[y] - p.s1_amount * trade1[i] + p.s2_amount * trade2[y] - p.s2_amount * trade2[i]) + p.money

            if (diff_trade[i] > (mean + std * thres)):

                if (p.s1_amount != 0):
                    pass

                else:
                    p.s1_amount -= trade1[i]/trade1[i]
                    p.s2_amount += trade1[i]/trade2[i]
                    y = i


            elif (diff_trade[i] < (mean - std*thres)):

                if (p.s1_amount != 0):
                    pass

                else:
                    p.s1_amount += trade2[i]/trade1[i]
                    p.s2_amount -= trade2[i]/trade2[i]
                    #money += trade2[i] - trade1[i]
                    y = i

            elif ((p.s1_amount < 0) and ((diff_trade[i]) <= mean)):


                print(y)
                print(i)
                print(trade1[y], trade1[i])
                print(trade2[y], trade2[i])

                print('s1y ' + str(-p.s1_amount * trade1[y]))
                print('s1i ' + str(p.s1_amount * trade1[i]))

                print('s2y ' + str(-p.s2_amount * trade2[y]))
                print('s2i ' + str(p.s2_amount * trade2[i]))

                p.open_days.append(i-y)

                p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] + p.s2_amount * trade2[y] - p.s2_amount * trade2[i]
                p.s1_amount = 0
                p.s2_amount = 0
                y = 0
                print('position closed')

            elif ((p.s1_amount > 0) and ((diff_trade[i]) >= mean)):

                print(y)
                print(i)

                print('s1y ' + str(-p.s1_amount * trade1[y]))
                print('s1i ' + str(p.s1_amount * trade1[i]))

                print('s2y ' + str(-p.s2_amount * trade2[y]))
                print('s2i ' + str(p.s2_amount * trade2[i]))


                print(trade1[y], trade1[i])
                print(trade2[y], trade2[i])

                p.open_days.append(i-y)

                p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] - p.s2_amount * trade2[i] + p.s2_amount * trade2[y]
                p.s1_amount = 0
                p.s2_amount = 0
                y = 0
                print('position closed')



            print(i, p.money, p.s1_amount, p.s2_amount)

            s = pd.Series({
                'Date': diff_trade.index[i],
                'money': p.money,
                'stock1': p.s1_amount,
                'stock2': p.s2_amount,
                'diff': diff_trade[i],
                'mean': mean,
                'portfolio_val': portfolio_val

            })



            df = pd.concat([df, s.to_frame().T], ignore_index = True)

        df = df.set_index(df['Date'])
        df = df.drop('Date', axis = 1)

        p.pf_value = df['portfolio_val']

        return p, df

    def distance_trading(self, stock1, stock2, thres):

        ## The distance_trading function takes ALL of the previous methods and combines them into one. Its returns are the portfolio
        ## and the the trade df which contains additional information regarding the trade distance, stock holdings and money evolution.

        thres = thres
        portfolio = Portfolio()

        mean, std = self.distance_trade_prep(stock1 = stock1, stock2 = stock2)

        diff_trade, trade1, trade2, mean, std, portfolio = self.pre_trade(stock1, stock2, mean, std, portfolio)

        portfolio, df = self.trading(diff_trade, trade1, trade2, mean, std, thres, portfolio)

        portfolio.sharpe_ratio()
        portfolio.average_open_days()

        return portfolio, df


    def perform_distance_trading(self):

        ## The final form of Trading: this method just uses the top 15% pairs with the least euclidean distance and
        ## returns it to the trading data dict.
        for i in range(int(len(self.distance_data)*0.15)):
            self.trading_data[i] = self.distance_trading(self.data[self.distance_data['stock1'][i]], self.data[self.distance_data['stock2'][i]], 1.5)


In [ ]:
class LDS_Trading:
    ## This is the first trading class for the thesis. It only supports Regular trading without the addition of a Stoploss or potential last day trading.
    ## Usecase of this class is the initial Start of the analysis for the individual timeframes. The functions within the class
    ## are explained where necessary.

    def __init__(self, start_date, end_date):
        ## setting start and end date of the crisis
        self.start_date = start_date
        self.end_date = end_date

        ## setting datapoints of the trading
        self.tickers = ''
        self.data = ''
        self.normed_data = ''

        self.trading_data = {}

        self.distance_data = ''
        self.run()


    def run(self):
        ## This method is used to auto-initialize the RegularTrading Class. With its help, we can focus on the important stuff - the analysis.

        self.tickers = self.get_ticker()
        self.data = self.get_ticker_data()
        self.normed_data = self.normer(self.data)
        self.distance_data = self.calc_distance(self.normed_data)
        self.perform_distance_trading()

    def get_ticker(self):

        ##This method is a function to return all stock picks. With its return of the Stock picks, we can easliy track
        ## which tickers have been traded.

        companies = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        tickers = companies[0].Symbol.to_list()
        random.seed(89146)
        stock_picks = random.sample(tickers, k = 25)
        return stock_picks

    def get_ticker_data(self):

        ## This is the download function to get all the ticker data from yfinance. For this usage, only the
        ## Adjusted Close will be downloaded.

        df = yf.download(self.tickers, self.start_date, self.end_date)['Adj Close']
        return df


    def train_split(self, data, length):

        ## This method is seperating the dataframe into the train period.

        train_period = pd.Series(dtype= 'float64')
        train_period = data.iloc[:int(len(data)*length)]
        return train_period

    def test_split(self, data, length):

        ## This method is seperating the dataframe into the test period.

        test_period = pd.Series(dtype= 'float64')
        test_period = data.iloc[int(len(data)*length):]
        return test_period

    def data_split(self, data, thres):
        ## This method takes the input data and splits it to two dataframes: train and test.
        train = self.train_split(data, thres)
        test = self.test_split(data, thres)
        return train, test

    def normer(self, data):

        ##The normer method (as the name suggests) divides the input data with the first price from the input timeseries and returns the normalized data"""
        normed_data = data/data.iloc[0]
        return normed_data

    def calc_distance(self, data):

        ## The calc_distance method calculates the euclidean distance from all timeseries of the normed data. It lists them in an ascending order
        ## in the dataframe - this is a crucial step in the processing of the output."""
        df = pd.DataFrame()
        for a1 in data.columns:
            for a2 in data.columns:

                if a1 != a2:

                    ## using the euclidean distance
                    diff = sum((data[a1] - data[a2])**2)

                    ## the calculated data will be appended to the dataframe with the series s below:

                    s = pd.Series({
                        'stock1': a1,
                        'stock2': a2,
                        'diff': diff
                    })

                    df = pd.concat([df, s.to_frame().T], ignore_index = True)

        df = df.drop_duplicates(subset = 'diff')
        df = df.sort_values(by = 'diff', ascending = True)
        df = df.reset_index()
        df.drop('index', axis = 1, inplace = True)

        return df

    def distance_trade_prep(self, stock1, stock2):

        ## This method prepares the input of the stocks (stock1 and stock2) for the further trading in the process. It will be used for the formation period
        ## of the trading. Therefore, the return of the method are the normed train data as well as the mean and the standard deviation.

        s = pd.Series(dtype= 'float64')


        train_s1, _ = self.data_split(stock1, 0.75)
        train_s2, _ = self.data_split(stock2, 0.75)

        normed_s1 = self.normer(train_s1)
        normed_s2 = self.normer(train_s2)

        s = normed_s1 - normed_s2

        mean = s.mean()
        std = s.std()

        return mean, std

    def pre_trade(self, stock1, stock2, mean, std, p: Portfolio()):

        ## This method will prepare everything for the trading itself. The returns are the difference of the trading data, the price
        ## data for the two stocks, the mean, std-deviation and the start portfolio which contains an
        ## stock1 amount of 0, stock2 amount of 0 and 0 money. This all is achived by combining the previous methods into the function.

        normed_s1 = self.normer(stock1)
        normed_s2 = self.normer(stock2)

        _, trade_normed1 = self.data_split(data = normed_s1,thres= 0.75)
        _, trade_normed2 = self.data_split(data = normed_s2, thres= 0.75)

        _, trade1 = self.data_split(data = stock1, thres = 0.75)
        _, trade2 = self.data_split(data = stock2, thres = 0.75)

        diff_trade = trade_normed1 - trade_normed2

        p.s1_name = stock1.name
        p.s2_name = stock2.name

        p.s1_amount = 0
        p.s2_amount = 0
        p.money = 0.01

        mean = mean
        std = std

        return diff_trade, trade1, trade2, mean, std, p



    def trading(self, diff_trade, trade1, trade2, mean, std, thres, p: Portfolio()):

        ## The Trading Function is spectacular unspectacular - it uses the provided inputs (stock1, stock2, pre-calculated mean, pre-calculated std,
        ## the threshold for the trading signals and the portfolio itself. Within the RegularTrading Class,
        ## the inclusion of the portfolio doesn't have o much impact, but in the later stages (and Classes) it will show its benefits.

        df = pd.DataFrame()

        thres = thres


        diff_trade = diff_trade

        mean = mean
        std = std
        thres = thres

        y = 0

        for i in range(len(diff_trade)):

            portfolio_val = -(p.s1_amount * trade1[y] - p.s1_amount * trade1[i] + p.s2_amount * trade2[y] - p.s2_amount * trade2[i]) + p.money

            if (diff_trade[i] > (mean + std * thres)):

                if (p.s1_amount != 0) and (i != len(diff_trade)-1):
                    pass

                elif ((i == len(diff_trade)-1) and (p.s1_amount != 0)):

                    p.open_days.append(i-y)

                    p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] - p.s2_amount * trade2[i] + p.s2_amount * trade2[y]
                    p.s1_amount = 0
                    p.s2_amount = 0
                    y = 0

                    print('last position closed '+ str(i))

                else:
                    p.s1_amount -= trade1[i]/trade1[i]
                    p.s2_amount += trade1[i]/trade2[i]
                    y = i


            elif (diff_trade[i] < (mean - std*thres)):

                if (p.s1_amount != 0) and (i != len(diff_trade)-1):
                    pass

                elif ((i == len(diff_trade)-1) and (p.s1_amount != 0)):

                    p.open_days.append(i-y)

                    p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] - p.s2_amount * trade2[i] + p.s2_amount * trade2[y]
                    p.s1_amount = 0
                    p.s2_amount = 0
                    y = 0
                    print('last position closed '+ str(i))

                else:
                    p.s1_amount += trade2[i]/trade1[i]
                    p.s2_amount -= trade2[i]/trade2[i]
                    #money += trade2[i] - trade1[i]
                    y = i

            elif ((p.s1_amount < 0) and ((diff_trade[i]) <= mean)):


                print(y)
                print(i)
                print(trade1[y], trade1[i])
                print(trade2[y], trade2[i])

                print('s1y ' + str(-p.s1_amount * trade1[y]))
                print('s1i ' + str(p.s1_amount * trade1[i]))

                print('s2y ' + str(-p.s2_amount * trade2[y]))
                print('s2i ' + str(p.s2_amount * trade2[i]))

                p.open_days.append(i-y)

                p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] + p.s2_amount * trade2[y] - p.s2_amount * trade2[i]
                p.s1_amount = 0
                p.s2_amount = 0
                y = 0
                print('position closed')

            elif ((p.s1_amount > 0) and ((diff_trade[i]) >= mean)):

                print(y)
                print(i)

                print('s1y ' + str(-p.s1_amount * trade1[y]))
                print('s1i ' + str(p.s1_amount * trade1[i]))

                print('s2y ' + str(-p.s2_amount * trade2[y]))
                print('s2i ' + str(p.s2_amount * trade2[i]))


                print(trade1[y], trade1[i])
                print(trade2[y], trade2[i])

                p.open_days.append(i-y)

                p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] - p.s2_amount * trade2[i] + p.s2_amount * trade2[y]
                p.s1_amount = 0
                p.s2_amount = 0
                y = 0
                print('position closed')



            print(i, p.money, p.s1_amount, p.s2_amount)

            s = pd.Series({
                'Date': diff_trade.index[i],
                'money': p.money,
                'stock1': p.s1_amount,
                'stock2': p.s2_amount,
                'diff': diff_trade[i],
                'mean': mean,
                'portfolio_val': portfolio_val

            })



            df = pd.concat([df, s.to_frame().T], ignore_index = True)

        df = df.set_index(df['Date'])
        df = df.drop('Date', axis = 1)

        p.pf_value = df['portfolio_val']

        return p, df

    def distance_trading(self, stock1, stock2, thres):

        ## The distance_trading function takes ALL of the previous methods and combines them into one. Its returns are the portfolio
        ## and the the trade df which contains additional information regarding the trade distance, stock holdings and money evolution.

        thres = thres
        portfolio = Portfolio()

        mean, std = self.distance_trade_prep(stock1 = stock1, stock2 = stock2)

        diff_trade, trade1, trade2, mean, std, portfolio = self.pre_trade(stock1, stock2, mean, std, portfolio)

        portfolio, df = self.trading(diff_trade, trade1, trade2, mean, std, thres, portfolio)

        portfolio.sharpe_ratio()
        portfolio.average_open_days()

        return portfolio, df


    def perform_distance_trading(self):

        ## The final form of Trading: this method just uses the top 15% pairs with the least euclidean distance and
        ## returns it to the trading data dict.
        for i in range(int(len(self.distance_data)*0.15)):
            self.trading_data[i] = self.distance_trading(self.data[self.distance_data['stock1'][i]], self.data[self.distance_data['stock2'][i]], 1.5)


In [ ]:
class EnhancedTrading:

    ## This is the last trading class for the thesis. It supports the enhanced Trading with the addition of a Stoploss or potential last day
    ## trading.
    ## Usecase of this class is the is the most advanced strat for the thesis. The functions within the class
    ## are explained where necessary.

    def __init__(self, start_date, end_date):
        ## setting start and end date of the crisis
        self.start_date = start_date
        self.end_date = end_date

        ## setting datapoints of the trading
        self.tickers = ''
        self.data = ''
        self.normed_data = ''

        self.trading_data = {}

        self.distance_data = ''


        self.run()


    def run(self):
        ## This method is used to auto-initialize the RegularTrading Class. With its help, we can focus on the important stuff - the analysis.

        self.tickers = self.get_ticker()
        self.data = self.get_ticker_data()
        self.normed_data = self.normer(self.data)
        self.distance_data = self.calc_distance(self.normed_data)
        self.perform_distance_trading()

    def get_ticker(self):

        ##This method is a function to return all stock picks. With its return of the Stock picks, we can easliy track
        ## which tickers have been traded.

        companies = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        tickers = companies[0].Symbol.to_list()
        random.seed(42)
        stock_picks = random.sample(tickers, k = 25)
        return stock_picks

    def get_ticker_data(self):

        ## This is the download function to get all the ticker data from yfinance. For this usage, only the
        ## Adjusted Close will be downloaded.

        df = yf.download(self.tickers, self.start_date, self.end_date)['Adj Close']
        return df


    def train_split(self, data, length):

        ## This method is seperating the dataframe into the train period.

        train_period = pd.Series(dtype= 'float64')
        train_period = data.iloc[:int(len(data)*length)]
        return train_period

    def test_split(self, data, length):

        ## This method is seperating the dataframe into the test period.

        test_period = pd.Series(dtype= 'float64')
        test_period = data.iloc[int(len(data)*length):]
        return test_period

    def data_split(self, data, thres):
        ## This method takes the input data and splits it to two dataframes: train and test.
        train = self.train_split(data, thres)
        test = self.test_split(data, thres)
        return train, test

    def normer(self, data):

        ##The normer method (as the name suggests) divides the input data with the first price from the input timeseries and returns the normalized data"""
        normed_data = data/data.iloc[0]
        return normed_data

    def calc_distance(self, data):

        ## The calc_distance method calculates the euclidean distance from all timeseries of the normed data. It lists them in an ascending order
        ## in the dataframe - this is a crucial step in the processing of the output."""
        df = pd.DataFrame()
        for a1 in data.columns:
            for a2 in data.columns:

                if a1 != a2:

                    ## using the euclidean distance
                    diff = sum((data[a1] - data[a2])**2)

                    ## the calculated data will be appended to the dataframe with the series s below:

                    s = pd.Series({
                        'stock1': a1,
                        'stock2': a2,
                        'diff': diff
                    })

                    df = pd.concat([df, s.to_frame().T], ignore_index = True)

        df = df.drop_duplicates(subset = 'diff')
        df = df.sort_values(by = 'diff', ascending = True)
        df = df.reset_index()
        df.drop('index', axis = 1, inplace = True)

        return df

    def distance_trade_prep(self, stock1, stock2):

        ## This method prepares the input of the stocks (stock1 and stock2) for the further trading in the process. It will be used for the formation period
        ## of the trading. Therefore, the return of the method are the normed train data as well as the mean and the standard deviation.

        s = pd.Series(dtype= 'float64')

        normed_s1 = self.normer(stock1)
        normed_s2 = self.normer(stock2)

        train_s1, _ = self.data_split(normed_s1, 0.75)
        train_s2, _ = self.data_split(normed_s2, 0.75)



        s = train_s1 - train_s2

        mean = s.mean()
        std = s.std()

        return mean, std

    def pre_trade(self, stock1, stock2, mean, std, p: Portfolio()):

        ## This method will prepare everything for the trading itself. The returns are the difference of the trading data, the price
        ## data for the two stocks, the mean, std-deviation and the start portfolio which contains an
        ## stock1 amount of 0, stock2 amount of 0 and 0 money. This all is achived by combining the previous methods into the function.

        normed_s1 = self.normer(stock1)
        normed_s2 = self.normer(stock2)

        _, trade_normed1 = self.data_split(data = normed_s1,thres= 0.75)
        _, trade_normed2 = self.data_split(data = normed_s2, thres= 0.75)

        _, trade1 = self.data_split(data = stock1, thres = 0.75)
        _, trade2 = self.data_split(data = stock2, thres = 0.75)

        diff_trade = trade_normed1 - trade_normed2

        p.s1_name = stock1.name
        p.s2_name = stock2.name

        p.s1_amount = 0
        p.s2_amount = 0
        p.money = 0.01

        mean = mean
        std = std

        return diff_trade, trade1, trade2, mean, std, p


    def stop_loss_trading(self, diff_trade, trade1, trade2, mean, std, thres, p: Portfolio, last_day_sell = None):

        """
        The Trading Function is spectacular unspectacular - it uses the provided inputs (stock1, stock2, pre-calculated mean, pre-calculate
        std, the threshold for the trading signals and the portfolio itself. The added portfolio from the classes before
        are really beneficial for the execution of the enhanced trading - the position sizes and money account are
        directly transferred to the new trading algorhithm.

        -------------
        Inputs:
        diff_trade: preprocessed trading difference between two stocks
        trade1: trading prices of stock1
        trade2: trading prices of stock2
        mean: precalculated mean of the training difference between the two prices
        std: precalculated std of the training difference between the two prices
        p: Portfolio, which contains all the current holdings
        last_day_sell: specifies whether all positions should be sold on the last trading day.

        -------------
        Returns:
        p: Portfolio with all current stock and money holdings
        df: Trading dataframe with (most) trading information

        """


        df = pd.DataFrame()

        thres = thres


        diff_trade = diff_trade
        trade1 = trade1
        trade2 = trade2


        mean = mean
        std = std
        thres = thres
        y = 0


        for i in range(len(diff_trade)):

            portfolio_val = -(p.s1_amount * trade1[y] - p.s1_amount * trade1[i] + p.s2_amount * trade2[y] - p.s2_amount * trade2[i]) + p.money


            if (diff_trade[i] > (mean + std*thres)) and (diff_trade[i] < (mean + std * (thres * 2))):

                if (p.s1_amount != 0) and (i != len(diff_trade)):
                    pass

                else:
                    p.s1_amount -= trade1[i]/trade1[i]
                    p.s2_amount += trade1[i]/trade2[i]

                    y = i

            elif ((p.s1_amount == 0) and (diff_trade[i] < (mean - std * (thres * 2))) or (diff_trade[i] > (mean + std * (thres * 2)))):
                pass

            elif (diff_trade[i] < (mean - std * thres)) and (diff_trade[i] > (mean - std * (thres * 2))):

                if (p.s1_amount != 0) and (i != len(diff_trade)):
                    pass

                else:
                    p.s1_amount += trade2[i]/trade1[i]
                    p.s2_amount -= trade2[i]/trade2[i]
                    y = i

            elif ((p.s1_amount < 0) and ((diff_trade[i]) <= mean)):

                p.open_days.append(i-y)

                p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] + p.s2_amount * trade2[y] - p.s2_amount * trade2[i]
                p.s1_amount = 0
                p.s2_amount = 0
                y = 0
                print('position closed')

            elif ((p.s1_amount > 0) and ((diff_trade[i]) >= mean)):

                p.open_days.append(i-y)

                p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] - p.s2_amount * trade2[i] + p.s2_amount * trade2[y]
                p.s1_amount = 0
                p.s2_amount = 0
                y = 0
                print('position closed')


            elif (diff_trade[i] < (mean - std * (thres * 2))) or (diff_trade[i] > (mean + std * (thres * 2))):

                p.open_days.append(i-y)

                p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] - p.s2_amount * trade2[i] + p.s2_amount * trade2[y]
                p.s1_amount = 0
                p.s2_amount = 0
                y = 0
                print('Position closed because of stop loss')



            elif last_day_sell == True:
                if ((i == len(diff_trade)-1) and (p.s1_amount != 0)):

                    p.open_days.append(i-y)

                    p.money -= p.s1_amount * trade1[y] - p.s1_amount * trade1[i] - p.s2_amount * trade2[i] + p.s2_amount * trade2[y]
                    p.s1_amount = 0
                    p.s2_amount = 0
                    y = 0

                    print('last position closed '+ str(i))

                else:
                    pass

            print(p.s1_amount, p.s2_amount, p.money)

            s = pd.Series({
                'Date': diff_trade.index[i],
                'money': p.money,
                'stock1': p.s1_amount,
                'stock2': p.s2_amount,
                'diff': diff_trade[i],
                'mean': mean,
                'portfolio_val': portfolio_val

            })


            df = pd.concat([df, s.to_frame().T], ignore_index = True)
        df = df.set_index(df['Date'])
        df = df.drop('Date', axis = 1)

        p.pf_value = df['portfolio_val']

        return p, df


    def enhanced_trading(self, diff_trade, trade1, trade2, mean, std, thres, p: Portfolio()):

        df1 = pd.DataFrame()

        p, df = self.stop_loss_trading(diff_trade, trade1, trade2, mean, std, thres, p)

        #if df['stock1'][-1] != 0 or df['stock2'][-1] != 0:
        #if p.s1_amount != 0:
        if (abs(df['diff'][-2]) > abs(df['diff'][-1])) and (p.s1_amount != 0):
            print('verlängerung hier hin')
            start1 = df.index[-1]
            end1 = start1 + datetime.timedelta(30)

            additional_1 = yf.download(p.s1_name, start = start1, end = end1)['Adj Close']
            additional_2 = yf.download(p.s2_name, start = start1, end = end1)['Adj Close']

            diff_trade_new = additional_1/self.data[p.s1_name][0] - additional_2/self.data[p.s2_name][0]
            diff_trade_new = normer(additional_1) - normer(additional_2)

            p, df1 = self.stop_loss_trading(diff_trade_new, additional_1, additional_2, mean, std, thres, p, last_day_sell = True)

            plt.plot(diff_trade_new)

        else:
            p.money = df['portfolio_val'][-1]
            p.s1_amount = 0
            p.s2_amount = 0


        plt.plot(diff_trade, label = str(p.s1_name + '_' + p.s2_name))
        plt.axhline(mean)
        plt.axhline(mean + std * thres)
        plt.axhline(mean + std*thres*2)
        plt.axhline(mean - std*thres)
        plt.axhline(mean - std*thres*2)
        plt.legend()



        df = pd.concat([df, df1], ignore_index=True)

        return p, df

    def distance_trading(self, stock1, stock2, thres):

        ## The distance_trading function takes ALL of the previous methods and combines them into one. Its returns are the portfolio
        ## and the the trade df which contains additional information regarding the trade distance, stock holdings and money evolution.

        thres = thres
        portfolio = Portfolio()


        mean, std = self.distance_trade_prep(stock1 = stock1, stock2 = stock2)

        diff_trade, trade1, trade2, mean, std, portfolio = self.pre_trade(stock1, stock2, mean, std, portfolio)

        portfolio, df = self.enhanced_trading(diff_trade, trade1, trade2, mean, std, thres, portfolio)
        portfolio.sharpe_ratio()
        portfolio.average_open_days()
        return portfolio, df


    def perform_distance_trading(self):

        ## The final form of Trading: this method just uses the top 15% pairs with the least euclidean distance and
        ## returns it to the trading data dict.
        for i in range(int(len(self.distance_data)*0.15)):
            self.trading_data[i] = self.distance_trading(self.data[self.distance_data['stock1'][i]], self.data[self.distance_data['stock2'][i]], 1.5)
